In [1]:
import gymnasium as gym
import mo_gymnasium as mo_gym
import numpy as np
import torch as th
import wandb
from torch import nn
from morl_baselines.common.networks import mlp
from morl_baselines.common.evaluation import log_episode_info
from morl_baselines.common.morl_algorithm import MOPolicy
from morl_baselines.single_policy.ser import mo_ppo


In [2]:

# Define the environment and parameters
env_id = "mo-halfcheetah-v4"  # Replace with the desired MO-Gymnasium environment
seed = 42
gamma = 0.995
steps_per_iteration = 2048
num_minibatches = 32
update_epochs = 10
learning_rate = 3e-4
clip_coef = 0.2
ent_coef = 0.0
vf_coef = 0.5
max_grad_norm = 0.5
gae_lambda = 0.95
device = th.device("cuda" if th.cuda.is_available() else "cpu")
weights = np.array([0.5, 0.5])  # Example weights for objectives


In [3]:
num_envs = 8
# Create vectorized environments
env_fns = [mo_ppo.make_env(env_id, seed + i, i, "run_name", gamma) for i in range(8)]
envs = gym.vector.SyncVectorEnv(env_fns)


c:\Users\thoma\anaconda3\envs\MORL_baseline\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.reward_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.reward_space` for environment variables or `env.get_wrapper_attr('reward_space')` that will search the reminding wrappers.
  logger.warn(
c:\Users\thoma\anaconda3\envs\MORL_baseline\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_wrapper_attr('num_envs')` that will search the reminding wrappers.
  logger.warn(
c:\Users\thoma\anaconda3\envs\MORL_baseline\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.is_vector_env to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.u

In [4]:
envs

SyncVectorEnv(8)

In [5]:
# Initialize the networks
obs_shape = envs.single_observation_space.shape
action_shape = envs.single_action_space.shape
# Get the reward dimension from the environment
temp_env = mo_ppo.make_env(env_id, seed, 0, "run_name", gamma)()
reward_dim = temp_env.reward_space.shape[0]

networks = mo_ppo.MOPPONet(obs_shape, action_shape, reward_dim)


In [6]:

# Initialize MOPPO
moppo = mo_ppo.MOPPO(
    id=0,
    networks=networks,
    weights=weights,
    envs=envs,
    log=True,
    steps_per_iteration=steps_per_iteration,
    num_minibatches=num_minibatches,
    update_epochs=update_epochs,
    learning_rate=learning_rate,
    gamma=gamma,
    anneal_lr=False,
    clip_coef=clip_coef,
    ent_coef=ent_coef,
    vf_coef=vf_coef,
    clip_vloss=True,
    max_grad_norm=max_grad_norm,
    norm_adv=True,
    target_kl=None,
    gae=True,
    gae_lambda=gae_lambda,
    device=device,
    seed=seed,
    rng=None,
)


In [7]:
import time

In [8]:

# Training loop
wandb.init(project="moppo_example")  # Initialize W&B logging
max_iterations = 1
start_time = time.time()

for iteration in range(1, max_iterations + 1):
    moppo.train(start_time, iteration, max_iterations)
    if iteration % 10 == 0:
        print(f"Iteration {iteration}/{max_iterations} completed.")

envs.close()
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: t-lautenbacher (thomas_lautenbacher). Use `wandb login --relogin` to force relogin


tensor([0.5000, 0.5000], dtype=torch.float64)
SPS: 1497
tensor([[-0.0016, -0.0266],
        [ 0.0083, -0.0305],
        [ 0.0120, -0.0238],
        [-0.0206, -0.0273],
        [-0.1042, -0.0269],
        [ 0.0302, -0.0296],
        [ 0.0797, -0.0264],
        [-0.0963, -0.0326]], dtype=torch.float64)


charts/SPS,▁
charts_0/learning_rate,▁
global_step,▁▁
losses_0/approx_kl,▁
losses_0/clipfrac,▁
losses_0/entropy,▁
losses_0/explained_variance,▁
losses_0/old_approx_kl,▁
losses_0/policy_loss,▁
losses_0/value_loss,▁
charts/SPS,1496
